# Elecciones presidenciales en Twitter

Vamos a tratar de evaluar la precencia de **Bots** en las campañas presidenciales de los dos candidatos en la segunda vuelta de las elecciones presidenciales chilenas en 2021. Para esto descargamos todos los Twits que mencionaran la palabra **Boric** o **Kast** durante la semana comprendida entre el 26.11.2021 y el 03.12.2021 inclusive. 

La infromacion obtenida se ve asi:

In [2]:
import pandas as pd

tweets_df = pd.read_json("tweets/all_tweets.json")
tweets_df

,created_at,id,author_id,text,entities
0,2021-12-01 21:27:49,1466157111570681856,3313242149,@rspg67 @abogadabruna @fbarrerarob @rob_i_c @C...,"{'mentions': [{'start': 0, 'end': 7, 'username..."
1,2021-12-01 21:27:49,1466157110824038400,1129003213204668416,RT @EduardoMadari13: MECHERO Tu le dijiste a J...,"{'mentions': [{'start': 3, 'end': 19, 'usernam..."
2,2021-12-01 21:27:49,1466157110668914688,1137473146020347904,@maylwino Señora vote por Kast para salvar al ...,"{'mentions': [{'start': 0, 'end': 9, 'username..."
3,2021-12-01 21:27:49,1466157109926584320,1309737469550698496,RT @kamiluru: @alicarina @0324Marta Desgraciad...,"{'mentions': [{'start': 3, 'end': 12, 'usernam..."
4,2021-12-01 21:27:49,1466157109536432128,142742893,RT @jenizaro11: No podia ser de otra forma! La...,"{'mentions': [{'start': 3, 'end': 14, 'usernam..."
...,...,...,...,...,...
2341605,2021-12-02 11:17:26,1466365891214381056,1047512358388748288,RT @Cooperativa: Diputado RN pro Kast declaró ...,"{'annotations': [{'start': 33, 'end': 36, 'pro..."
2341606,2021-12-02 11:17:26,1466365890270801920,708339048495063040,RT @NYC_prensa: [KAST] “El país no se construy...,"{'annotations': [{'start': 60, 'end': 73, 'pro..."
2341607,2021-12-02 11:17:26,1466365889687670784,262972427,RT @aprachile: ✅José Antonio Kast durante gira...,"{'annotations': [{'start': 16, 'end': 32, 'pro..."
2341608,2021-12-02 11:17:25,1466365885849882624,224427268,RT @jimenaorrego: Kast se juntó en USA con el ...,"{'annotations': [{'start': 18, 'end': 21, 'pro..."


In [6]:
pd.to_timedelta(tweets_df.created_at.max() - tweets_df.created_at.min(), unit='D')

Timedelta('7 days 04:50:29')

In [8]:
tweets_df.created_at.apply(lambda x: x.day).unique()

array([ 1, 30, 27, 26, 29, 28,  3,  2])

## 1. Hashtags mas populares

Partimeros nuestro analsis contandi los hashtags usados por los usarios en sus Twitts. Para limitar el analisis a menciones directas a los candidatos, solo estudiaremos hashtags que incluyan las palabras **Boric** o **Kast**:


In [9]:
def top_hashtags(df):
    hashtags_dict = {}
    for index, values in df.entities.items():
        if values and ('hashtags' in values):
            hashtags = values['hashtags']       
            for hashtag in hashtags:
                # check if hashtags is in the dictionary
                tag = hashtag['tag']
                if ("boric" in tag.lower()) or ("kast" in tag.lower()):
                    if tag in hashtags_dict:
                        hashtags_dict[tag] += 1
                    else:
                        hashtags_dict[tag] = 1
    hashtags_df = pd.DataFrame.from_dict(hashtags_dict, orient='index', columns=['count'])
    return hashtags_df.sort_values(by='count', ascending=False)

In [13]:
top_hashtags_df = top_hashtags(tweets_df)
top_hashtags_df.head(20)

,count
Boric,45269
BoricPresidente,35257
Kast,31339
MujeresPorKast,21976
TodoChileVotaKast,19090
KastPresidente2022,17759
TodosKast,12656
BoricPresidente2022,7749
BoricNoSeAtreveDebatir,7065
kast,6617


En esta tabla tenemos los hashtags y cuantas veces se usaron. La tabla esta ordenada por popularidad y de manera decreciente. Para nuestro analisis excluiremos los hashtags #Kast, #Boric , #kast y #boric. Esto dado que tanto un comentario positivo como negativo al respecto de los candidatos puede incluirlos y es muy dificil clasificar a quien apoyan. 

La lista de los 10 hashtags mas populares excluyendo los hastags mencionados se ve asi:

In [64]:
hashtags_list = ['BoricPresidente',
 'MujeresPorKast',
 'TodoChileVotaKast',
 'KastPresidente2022',
 'TodosKast',
 'BoricPresidente2022',
 'BoricNoSeAtreveDebatir',
 'BoricNoSeLaPuede',
 'BoricMiente',
 'KastPresidente']

top_hashtags_df = top_hashtags_df[top_hashtags_df.index.isin(hashtags_list)]
top_hashtags_df

,count
BoricPresidente,35257
MujeresPorKast,21976
TodoChileVotaKast,19090
KastPresidente2022,17759
TodosKast,12656
BoricPresidente2022,7749
BoricNoSeAtreveDebatir,7065
BoricNoSeLaPuede,5610
BoricMiente,4733
KastPresidente,4725


## 2. Identificar si una cuenta es un Bot o una persona real

Primero debemos definir que vamos a llamar Bot. Para esto usamos un modelo llamado Botometer. Este modelo es desarollado por el ["Observatory on Social Media"](https://osome.iu.edu/), el cual es un proyecto conjunto del "Center for Complex Networks and Systems Research" y el "Network Science Institute" de la Universidad de Indiana de Estados Unidos.

Toda la informacion referente a la metodologia la pueden encontrar [aqui](https://osome.iu.edu/research/publications)

Basicamente el modelo tiene como input el `id` de una cuenta de Twitter. El programa revisa varias caracteristicas de la cuenta en evaluacion: frecuencia de poste, red de contactos, vocabulario usado, consulta a bases de datos con informacion previa sobre cuentas identificadas como bots, etc. Con esta informacion el modelo entrega varios parametros que describen la actividad de la cuenta y le asigna una probabilidad a que la cuenta sea o no un Bot

Nuestro criterio para clasificar una cuenta como bot es que el modelo le asigne una probabilidad general mayor a un 50%. Esta probabilidad es la columna `universal` en la tabla siguiente:

In [21]:
users_df = pd.read_json("data/clean/users.json").drop("english", axis=1)
users_df

,id_str,screen_name,universal,status
0,1121505338690293760,None,NaN,deleted
1,1313512294815539200,KarlPop78534593,0.45,active
2,1465407053170724864,None,NaN,deleted
3,1307838136010117120,None,NaN,deleted
4,345987090,Lina1061,0.43,active
...,...,...,...,...
30722,1408048859041964032,None,NaN,deleted
30723,219678666,UnPerrit0,0.17,active
30724,68507689,Genghis__Khan,0.07,active
30725,789919502859403264,None,NaN,deleted


Lo primero que nos llamo la atencion fue que muchas cuentas ya no existen. Las cuentas dejan de existir por 3 razones:
- El usuario voluntariamente elimino su cuenta
- Twitter bloqueo la cuenta por violacion a los terminos de uso
- La cuenta ha estado inactivo por un largo periodo de tiempo 

La ultima razon la podemos descartar, dado que el analisis fue hecho posteriormente a la generacion de un Tweet.

La tasa de cuentas **borradas** es la siguiente:

In [85]:
usuarios_activos = len(users_df[users_df.status == "active"])
usuarios_inactivos = len(users_df[users_df.status == "deleted"])
ratio_borrados = usuarios_inactivos / (usuarios_inactivos + usuarios_activos)

print("Usuarios activos:", usuarios_activos)
print("Usuarios borrados:", usuarios_inactivos)
print(f"Proporcion de usuarios borrados: {(ratio_borrados * 100):.2f}%")

Usuarios activos: 22663
Usuarios borrados: 8064
Proporcion de usuarios borrados: 26.24%


Por otro lado, una vista general a las cuentas clasificadas como bots:

In [47]:
bots_df = users_df[(users_df.status == "active") & (users_df.universal > 0.5)]
print("Bots:", len(bots_df))
print(f"Proporcion de bots sobre usuarios activos: {(len(bots_df)/usuarios_activos):.2f}%")

Bots: 3303
Proporcion de bots sobre usuarios activos: 0.15%


Para desglozar esta informacion vamos a asignar los hashtags segun su apoyo: 

In [51]:
hashtags_support_dict = {
 "BoricPresidente": "Boric",
 "MujeresPorKast": "Kast",
 "TodoChileVotaKast": "Kast",
 "KastPresidente2022": "Kast",
 "TodosKast": "Kast",
 "BoricPresidente2022": "Boric",
 "BoricNoSeAtreveDebatir": "Kast",
 "BoricNoSeLaPuede": "Kast",
 "BoricMiente": "Kast",
 "KastPresidente": "Kast"
 }

Cuantos tweets contienen hashtags que apoyan a cada candidato?

In [68]:
top_hashtags_df["support"] = top_hashtags_df.apply(lambda x: hashtags_support_dict[x.name], axis=1)
top_hashtags_df

/var/folders/3v/y9lrkp790dl4phjfd0r91k400000gn/T/ipykernel_27130/2123882457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_hashtags_df["support"] = top_hashtags_df.apply(lambda x: hashtags_support_dict[x.name], axis=1)


,count,support
BoricPresidente,35257,Boric
MujeresPorKast,21976,Kast
TodoChileVotaKast,19090,Kast
KastPresidente2022,17759,Kast
TodosKast,12656,Kast
BoricPresidente2022,7749,Boric
BoricNoSeAtreveDebatir,7065,Kast
BoricNoSeLaPuede,5610,Kast
BoricMiente,4733,Kast
KastPresidente,4725,Kast


In [86]:
tweets_kast = top_hashtags_df[top_hashtags_df.support == "Kast"]["count"].sum()
tweets_boric = top_hashtags_df[top_hashtags_df.support == "Boric"]["count"].sum()

kast_boric_ratio = tweets_kast / tweets_boric

print("Tweets apoyando a Kast:", tweets_kast)
print("Tweets apoyando a Boric:", tweets_boric)

print(f"Ratio de tweets Kast/Boric: {kast_boric_ratio:.2f}")

Tweets apoyando a Kast: 93614
Tweets apoyando a Boric: 43006
Ratio de tweets Kast/Boric: 2.18


Aqui podemos ver que Kast tiene 2.18 veces el numero de tweets a su favor que Boric

### Bots por hashtag

Revisemos primero cuantas cuentas borradas tiene cada hashtag.

In [145]:
# users per hashtag
hashtags_df = pd.DataFrame(columns=["hashtag", "active", "deleted", "bots"])

for hashtag in hashtags_list:
    active_users = 0
    deleted_users = 0
    bot_users = 0
    hashtag_df = tweets_df[tweets_df.text.str.contains(hashtag)]
    hashtag_users_list = list(hashtag_df.author_id.unique())
    for user in hashtag_users_list:
        users_info_df = users_df[users_df.id_str == user]
        if users_info_df.status.values[0] == "active":
            active_users += 1
        if users_info_df.status.values[0] == "deleted":
            deleted_users += 1
        if (users_info_df.universal > 0.5).values[0]:
            bot_users += 1
    hashtags_df = hashtags_df.append({
        "hashtag": hashtag,
        "active": active_users,
        "deleted": deleted_users,
        "bots": bot_users
    }, ignore_index=True)

hashtags_df

,hashtag,active,deleted,bots
0,BoricPresidente,12841,3930,1158
1,MujeresPorKast,4489,2041,1494
2,TodoChileVotaKast,4497,2061,1549
3,KastPresidente2022,4742,2115,1571
4,TodosKast,3823,1760,1393
5,BoricPresidente2022,3036,984,193
6,BoricNoSeAtreveDebatir,2342,1078,834
7,BoricNoSeLaPuede,2567,1145,988
8,BoricMiente,3515,1558,1105
9,KastPresidente,5537,2521,1749


In [148]:
hashtags_df = hashtags_df.merge(top_hashtags_df, how="left", left_on="hashtag", right_index=True)
hashtags_df

,hashtag,active,deleted,bots,count,support
0,BoricPresidente,12841,3930,1158,35257,Boric
1,MujeresPorKast,4489,2041,1494,21976,Kast
2,TodoChileVotaKast,4497,2061,1549,19090,Kast
3,KastPresidente2022,4742,2115,1571,17759,Kast
4,TodosKast,3823,1760,1393,12656,Kast
5,BoricPresidente2022,3036,984,193,7749,Boric
6,BoricNoSeAtreveDebatir,2342,1078,834,7065,Kast
7,BoricNoSeLaPuede,2567,1145,988,5610,Kast
8,BoricMiente,3515,1558,1105,4733,Kast
9,KastPresidente,5537,2521,1749,4725,Kast


### Resumen por candidato y hashtag

In [149]:
kast_df = hashtags_df[hashtags_df.support == "Kast"]
kast_df

,hashtag,active,deleted,bots,count,support
1,MujeresPorKast,4489,2041,1494,21976,Kast
2,TodoChileVotaKast,4497,2061,1549,19090,Kast
3,KastPresidente2022,4742,2115,1571,17759,Kast
4,TodosKast,3823,1760,1393,12656,Kast
6,BoricNoSeAtreveDebatir,2342,1078,834,7065,Kast
7,BoricNoSeLaPuede,2567,1145,988,5610,Kast
8,BoricMiente,3515,1558,1105,4733,Kast
9,KastPresidente,5537,2521,1749,4725,Kast


In [159]:
kast_active_count = kast_df.active.sum()
kast_deleted_count = kast_df.deleted.sum()
kast_bots_count = kast_df.bots.sum()
kast_tweets_count = kast_active_count + kast_deleted_count + kast_bots_count

kast_active_ratio = kast_active_count / kast_tweets_count
kast_deleted_ratio = kast_deleted_count / kast_tweets_count
kast_bots_ratio = kast_bots_count / kast_tweets_count

print("Tweets a favor de Kast:", kast_tweets_count)
print(f"Proporcion de tweets de usuarios activos: {(kast_active_ratio * 100):.2f}%")
print(f"Proporcion de tweets de usuarios eliminados: {(kast_deleted_ratio * 100):.2f}%")
print(f"Proporcion de tweets de bots: {(kast_bots_ratio * 100):.2f}%")



Tweets a favor de Kast: 56474
Proporcion de tweets de usuarios activos: 55.80%
Proporcion de tweets de usuarios eliminados: 25.28%
Proporcion de tweets de bots: 18.92%


In [160]:
boric_df = hashtags_df[hashtags_df.support == "Boric"]
boric_df

,hashtag,active,deleted,bots,count,support
0,BoricPresidente,12841,3930,1158,35257,Boric
5,BoricPresidente2022,3036,984,193,7749,Boric


In [161]:
boric_active_count = boric_df.active.sum()
boric_deleted_count = boric_df.deleted.sum()
boric_bots_count = boric_df.bots.sum()
boric_tweets_count = boric_active_count + boric_deleted_count + boric_bots_count

bric_active_ratio = boric_active_count / boric_tweets_count
boric_deleted_ratio = boric_deleted_count / boric_tweets_count
boric_bots_ratio = boric_bots_count / boric_tweets_count

print("Tweets a favor de Boric:", boric_tweets_count)
print(f"Proporcion de tweets de usuarios activos: {(bric_active_ratio * 100):.2f}%")
print(f"Proporcion de tweets de usuarios eliminados: {(boric_deleted_ratio * 100):.2f}%")
print(f"Proporcion de tweets de bots: {(boric_bots_ratio * 100):.2f}%")

Tweets a favor de Boric: 22142
Proporcion de tweets de usuarios activos: 71.71%
Proporcion de tweets de usuarios eliminados: 22.19%
Proporcion de tweets de bots: 6.10%
